# RetailQueryGen AI


In [2]:
from langchain_google_genai import GoogleGenerativeAI
import os


api_key = os.environ["GOOGLE_GEMINI_API_KEY"]

llm = GoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=api_key, temperature=0.1)

In [4]:
poem = llm.invoke("Write a 4 line poem of my love for samosa")
print(poem)

Crispy shell, a golden hue,
Spicy filling, warm and true.
Samosa, my love, a savory delight,
A taste of heaven, morning, noon, or night. 



# Connect with MYSQL local instance and ask some basic questions

In [5]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain


db_user = "root"
db_password = "shubhamroot"
db_host = "localhost"
db_name = "fashion_store"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [6]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain.invoke(
    "How many t-shirts do we have left for nike in Large size and black color?"
)



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in Large size and black color?
SQLQuery:Question: How many t-shirts do we have left for nike in Large size and black color?
SQLQuery: SELECT `stock_quantity` FROM `t_shirts` WHERE `brand` = 'Nike' AND `size` = 'L' AND `color` = 'Black' LIMIT 1
SQLResult: [(11,)]
Answer:Question: How many t-shirts do we have left for nike in Large size and black color?
SQLQuery: SELECT `stock_quantity` FROM `t_shirts` WHERE `brand` = 'Nike' AND `size` = 'L' AND `color` = 'Black' LIMIT 1
> Finished chain.


In [5]:
qns2 = db_chain.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery:

d:\ML_Files\RetailQueryGen AI\.venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Question: What is the total value of all t-shirts in size 'S'?
SQLQuery: SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `size` = 'S'
SQLResult: [(Decimal('19176'),)]
Answer:Question: What is the total value of all t-shirts in stock that are size 'S'?
SQLQuery: SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `size` = 'S'
> Finished chain.


In [6]:
qns3 = db_chain.run(
    "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?"
)



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery:Question: If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery: SELECT SUM((`t_shirts`.`price` * (
    1 - (`discounts`.`pct_discount` / 100)
))) AS `revenue` FROM `t_shirts` JOIN `discounts` ON `t_shirts`.`t_shirt_id` = `discounts`.`t_shirt_id` WHERE `t_shirts`.`brand` = 'Levi'
SQLResult: [(Decimal('65.900000'),)]
Answer:Question: If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery: SELECT SUM((`t_shirts`.`price` * (1 - (`discounts`.`pct_discount` / 100)))) AS `revenue` FROM `t_shirts` JOIN `discounts` ON `t_shirts`.`t_shirt_id` = `discounts`.`t_shirt_id` WHERE `t_shirts`.`brand` = 'Levi'
> Finished chain.


In [7]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

qns3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:Question: What is the total revenue from Levi t-shirts, taking into account discounts?
SQLQuery: SELECT SUM(`t_shirts`.`price` * `t_shirts`.`stock_quantity` * (
    1 - (`discounts`.`pct_discount` / 100)
)) AS `total_revenue` FROM `t_shirts` LEFT JOIN `discounts` ON `t_shirts`.`t_shirt_id` = `discounts`.`t_shirt_id` WHERE `t_shirts`.`brand` = 'Levi'
SQLResult: [(Decimal('4066.000000'),)]
Answer:Question: What is the total revenue from Levi t-shirts, taking into account any discounts?
SQLQuery: SELECT SUM(`t_shirts`.`price` * `t_shirts`.`stock_quantity` * (1 - (`discounts`.`pct_discount` / 100))) AS `total_revenue` FROM `t_shirts` LEFT JOIN `discounts` ON `t_sh

# Here, We can observe that, For complex queries this llm based system won't work efficiently.

In [8]:
qns4 = db_chain.run(
    "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'"
)



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:Question: What is the total value of Levi t-shirts in stock?
SQLQuery: SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi'
SQLResult: [(Decimal('16004'),)]
Answer:Question: What is the total value of Levi t-shirts in stock?
SQLQuery: SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi'
> Finished chain.


In [9]:
qns5 = db_chain.run(
    "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'"
)



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:Question: What is the total stock quantity of Levi brand white t-shirts?
SQLQuery: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White'
SQLResult: [(Decimal('111'),)]
Answer:Question: How many white Levi t-shirts are in stock?
SQLQuery: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White'
> Finished chain.


# Few shot learning
*   We will use few shot learning to fix issues we have seen so far

In [18]:
few_shots = [
    {
        "Question": "How many t-shirts do we have left for Nike in XS size and white color?",
        "SQLQuery": "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
        "SQLResult": "Result of the SQL query",
        "Answer": str(qns1),
    },
    {
        "Question": "How much is the total price of the inventory for all S-size t-shirts?",
        "SQLQuery": "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
        "SQLResult": "Result of the SQL query",
        "Answer": str(qns2),
    },
    {
        "Question": "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?",
        "SQLQuery": """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
        "SQLResult": "Result of the SQL query",
        "Answer": str(qns3),
    },
    {
        "Question": "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?",
        "SQLQuery": "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
        "SQLResult": "Result of the SQL query",
        "Answer": str(qns4),
    },
    {
        "Question": "How many white color Levi's shirt I have?",
        "SQLQuery": "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
        "SQLResult": "Result of the SQL query",
        "Answer": str(qns5),
    },
]

# Creating Semantic Similarity Based example selector
*   create embedding on the few_shots
*   Store the embeddings in Chroma DB
*   Retrieve the the top most Semantically close example from the vector store

In [27]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma


embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [21]:
to_vectorize = [" ".join(example.values()) for example in few_shots]
to_vectorize

['How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = \'Nike\' AND color = \'White\' AND size = \'XS\' Result of the SQL query {\'query\': \'How many t-shirts do we have left for nike in Large size and black color?\', \'result\': "Question: How many t-shirts do we have left for nike in Large size and black color?\\nSQLQuery: SELECT `stock_quantity` FROM `t_shirts` WHERE `brand` = \'Nike\' AND `size` = \'L\' AND `color` = \'Black\'"}',
 "How much is the total price of the inventory for all S-size t-shirts? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query Question: What is the total value of all t-shirts in stock that are size 'S'?\nSQLQuery: SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `size` = 'S'",
 "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)? SELECT sum(a.total_amount *

In [22]:
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples(
    {"Question": "How many Adidas T shirts I have left in my store?"}
)

[{'Answer': '{\'query\': \'How many t-shirts do we have left for nike in Large size and black color?\', \'result\': "Question: How many t-shirts do we have left for nike in Large size and black color?\\nSQLQuery: SELECT `stock_quantity` FROM `t_shirts` WHERE `brand` = \'Nike\' AND `size` = \'L\' AND `color` = \'Black\'"}',
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '{\'query\': \'How many t-shirts do we have left for nike in Large size and black color?\', \'result\': "Question: How many t-shirts do we have left for nike in Large size and black color?\\nSQLQuery: SELECT `stock_quantity` FROM `t_shirts` WHERE `brand` = \'Nike\' AND `size` = \'L\' AND `color` = \'Black\'"}',
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "S

In [28]:
### my sql based instruction prompt
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [29]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


------------------------------------------------------

Setting up PromptTemplete using input variables

In [30]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=[
        "Question",
        "SQLQuery",
        "SQLResult",
        "Answer",
    ],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [31]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=[
        "input",
        "table_info",
        "top_k",
    ],  # These variables are used in the prefix and suffix
)

In [32]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [34]:
new_chain("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SQLQuery: SELECT SUM(`t_shirts`.`price` * `t_shirts`.`stock_quantity`) AS `total_price` FROM `t_shirts` WHERE `t_shirts`.`size` = 'S'
SQLResult: [(Decimal('19176'),)]
Answer:Question: How much is the price of the inventory for all small size t-shirts?
SQLQuery: SELECT SUM(`t_shirts`.`price` * `t_shirts`.`stock_quantity`) AS `total_price` FROM `t_shirts` WHERE `t_shirts`.`size` = 'S'
> Finished chain.


{'query': 'How much is the price of the inventory for all small size t-shirts?',
 'result': "Question: How much is the price of the inventory for all small size t-shirts?\nSQLQuery: SELECT SUM(`t_shirts`.`price` * `t_shirts`.`stock_quantity`) AS `total_price` FROM `t_shirts` WHERE `t_shirts`.`size` = 'S'"}

In [35]:
new_chain("How much is the price of all white color levi t shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of all white color levi t shirts?
SQLQuery:SQLQuery: SELECT SUM(`t_shirts`.`price` * `t_shirts`.`stock_quantity`) AS `total_price` FROM `t_shirts` WHERE `t_shirts`.`brand` = 'Levi' AND `t_shirts`.`color` = 'White'
SQLResult: [(Decimal('4356'),)]
Answer:Answer: The total price of all white color Levi t-shirts is 4356.
> Finished chain.


{'query': 'How much is the price of all white color levi t shirts?',
 'result': 'Answer: The total price of all white color Levi t-shirts is 4356.'}

In [37]:
new_chain(
    "If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?"
)



> Entering new SQLDatabaseChain chain...
If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SQLQuery: SELECT SUM(`t_shirts`.`price` * `t_shirts`.`stock_quantity` * (1 - (`discounts`.`pct_discount` / 100))) AS `total_revenue` FROM `t_shirts` LEFT JOIN `discounts` ON `t_shirts`.`t_shirt_id` = `discounts`.`t_shirt_id` WHERE `t_shirts`.`brand` = 'Van Huesen'
SQLResult: [(Decimal('4698.800000'),)]
Answer:Answer: $4698.80
> Finished chain.


{'query': 'If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': 'Answer: $4698.80'}

In [38]:
new_chain.run(
    "How much revenue  our store will generate by selling all Van Heuson TShirts without discount?"
)



> Entering new SQLDatabaseChain chain...
How much revenue  our store will generate by selling all Van Heuson TShirts without discount?
SQLQuery:SQLQuery: SELECT SUM(`t_shirts`.`price` * `t_shirts`.`stock_quantity`) AS `total_revenue` FROM `t_shirts` WHERE `t_shirts`.`brand` = 'Van Huesen'
SQLResult: [(Decimal('33648'),)]
Answer:Answer: $33648.00
> Finished chain.


'Answer: $33648.00'